In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.utils import resample
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.corpus import stopwords

# Download the necessary NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')

# Define a list of stop words
stop_words = set(stopwords.words('english'))

# Load the Amazon reviews dataset
data_en = pd.read_pickle(r"C:\Users\A9236\Data\data_en.pickle")

# Combine the review headline and body into a single text column
data_en["Review"] = data_en["review_headline"] + " " + data_en["review_body"]

# Define the input and output data
X = data_en["Review"]
y = data_en["star_rating"]

# Upsample the minority classes to balance the dataset
rs = [resample(X[y == sr], y[y == sr], replace=False, n_samples=10000, random_state=123) for sr in [1,2,3,4,5]]
X_list = [rs[r][0] for r in range(5)]
y_list = [rs[r][1] for r in range(5)]
X_us = np.hstack(X_list)
y_us = np.hstack(y_list)

# Split the data into training, validation, and test sets
X_train, X_test_val, y_train, y_test_val = train_test_split(X_us, y_us, test_size= 0.4, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test_val, y_test_val, random_state=42)

# Tokenize the reviews into individual words and remove stop words
X_train_tokens = [[word for word in word_tokenize(review) if word.lower() not in stop_words] for review in X_train]
X_test_tokens = [[word for word in word_tokenize(review) if word.lower() not in stop_words] for review in X_test]

# Perform POS tagging on the tokenized reviews
X_train_pos = [pos_tag(tokens) for tokens in X_train_tokens]
X_test_pos = [pos_tag(tokens) for tokens in X_test_tokens]

# Create the input data for the word2vec model
reviews = X_train_pos + X_test_pos
sentences = []
for review in reviews:
    sentence = []
    for word, tag in review:
        sentence.append(word.lower() + '_' + tag)
    sentences.append(sentence)

# Train a word2vec model using TensorFlow
embedding_dim = 100
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(input_dim=10000, output_dim=embedding_dim, input_length=50),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(5)
])
model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(np.array([np.array([word2vec_model.wv[w] for w in s if w in word2vec_model.wv])[:50] for s in sentences]), np.array(y_train), epochs=10, validation_data=(np.array([np.array([word2vec_model.wv[w] for w in s if w in word2vec_model.wv])[:50] for s in X_test_pos]), np.array(y_test)))

ImportError: Traceback (most recent call last):
  File "C:\Users\A9236\AppData\Local\anaconda3\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 62, in <module>
    from tensorflow.python._pywrap_tensorflow_internal import *
ImportError: DLL load failed while importing _pywrap_tensorflow_internal: Operation did not complete successfully because the file contains a virus or potentially unwanted software.


Failed to load the native TensorFlow runtime.
See https://www.tensorflow.org/install/errors for some common causes and solutions.
If you need help, create an issue at https://github.com/tensorflow/tensorflow/issues and include the entire stack trace above this error message.